In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torchsummary import summary
from tqdm import tqdm

In [2]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device: str
if use_cuda:
    device = "cuda"
elif use_mps:
    device = "mps"
else:
    device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.20),
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.20),
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout(0.20),
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=1)

    def summarize(self):
        summary(self, input_size=(1, 28, 28))

In [4]:
Net().summarize()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             584
              ReLU-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
         MaxPool2d-7            [-1, 8, 14, 14]               0
           Dropout-8            [-1, 8, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           1,168
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 16, 14, 14]           2,320
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1, 16,

In [5]:
torch.manual_seed(1)
batch_size = 64
kwargs = {"num_workers": 1, "pin_memory": True} if device == "cuda" else {}

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
])

# Download and create datasets
train_dataset = datasets.MNIST(
    root='../data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = datasets.MNIST(
    root='../data', 
    train=False, 
    download=True, 
    transform=transform
)

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,
    **kwargs,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=1000, 
    shuffle=False,
    **kwargs,
)

In [6]:
def train_step(epoch, model, device, train_loader, optimizer):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        
        pbar.set_description(desc=f"epoch={epoch:02d} loss={loss.item():.4f} batch_id={batch_idx:04d} accuracy={100*correct/processed:.2f}%")


def test_step(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="mean").item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)  # Average loss across batches

    print(
        "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

In [7]:
epochs = 20
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, epochs + 1):
    train_step(epoch, model, device, train_loader, optimizer)
    test_step(model, device, test_loader)

epoch=01 loss=0.3911 batch_id=0937 accuracy=83.33%: 100%|██████████| 938/938 [00:08<00:00, 110.16it/s]


Test set: Average loss: 0.0567, Accuracy: 9829/10000 (98.29%)



epoch=02 loss=0.3151 batch_id=0937 accuracy=86.32%: 100%|██████████| 938/938 [00:08<00:00, 114.43it/s]


Test set: Average loss: 0.0370, Accuracy: 9884/10000 (98.84%)



epoch=03 loss=0.4425 batch_id=0937 accuracy=86.77%: 100%|██████████| 938/938 [00:08<00:00, 110.87it/s]


...

Test set: Average loss: 0.0227, Accuracy: 9920/10000 (99.20%)



epoch=19 loss=0.1358 batch_id=0937 accuracy=87.95%: 100%|██████████| 938/938 [00:08<00:00, 115.15it/s]


Test set: Average loss: 0.0217, Accuracy: 9926/10000 (99.26%)



epoch=20 loss=0.5194 batch_id=0937 accuracy=87.83%: 100%|██████████| 938/938 [00:08<00:00, 114.67it/s]


Test set: Average loss: 0.0346, Accuracy: 9903/10000 (99.03%)

